   # 📜 Analytical Project 📜 | E - COMMERCE |

   #### owner: Golovin Alexey
   
***

### Project Task

1. How many users have made a single purchase?

2. On average, how many orders are not delivered due to various reasons (Make a detailed breakdown of reasons)? 

3. For each product, determine the day of the week on which the product is most frequently bought

4. What is the average number of purchases per customer per week (in month)? Please note that within a month, the number of weeks may not be a whole number. For example, November 2021 had 4.28 weeks

5. Using pandas, perform a cohort analysis of customers for the period from January to December and identify the cohort(s) with the highest retention for the 3rd month.

6. "Frequently, for high-quality analysis of audience, segmentation-based methods are used. Use Python to perform RFM segmentation of customers in order to make a high-quality evaluation of the audience. For clustering, you can use the following metrics: R (Recency) - time elapsed since the last purchase before the current date, F (Frequency) - total number of purchases made over time, and M (Monetary) - total amount spent over time.

### List of datasets:

<b>olist_customers_datase.csv</b> — dataset of unique customer <br>
<i>customer_id </i> — id of customer for each order <br> 
<i>customer_unique_id </i> —  unique id of customer <br>
<i>customer_zip_code_prefix </i>—  zip-code  <br>
<i>customer_city </i> —  city of delivery <br>
<i>customer_state</i> —  state of delivery <br>

<b>olist_orders_dataset.csv </b> —  orders dataset <br>
<i>order_id</i> —  unique order id (check) <br>
<i>customer_id</i> —  id of customer for each order <br>
<i>order_status</i> —  order status <br>
<i>order_purchase_timestamp</i> —  time/date of purchase <br>
<i>order_approved_at</i> —  time/date of approving order <br>
<i>order_delivered_carrier_date</i> — time/date of issuance for logistic-partner <br>
<i>order_delivered_customer_date</i> — time/date of delivery <br>
<i>order_estimated_delivery_date</i> — estimated_delivery time/date  <br>

<b>olist_order_items_dataset.csv</b> —  items in order <br>
<i>order_id</i> —  unique order id (check) <br>
<i>order_item_id</i> —  id inside one order <br>
<i>product_id</i> —  product id (barcode) <br>
<i>seller_id</i> — id of distributor <br>
<i>shipping_limit_date</i> —  maximum date for delivery, before order will be giving to logistic-partner <br>
<i>price</i> —  price per unit <br>
<i>freight_value </i> — weight of product <br>


Example of structure of data, you can visualise on <b>order_id</b> == `00143d0f86d6fbd9f9b38ab440ac16f5`  <br>


Unique statuses for orders in dataset *olist_orders_dataset*: <br>

* created  <br>
* approved  <br>
* invoiced  <br>
* processing <br>
* shipped <br>
* delivered <br>
* unavailable <br>
* canceled<br>


At first, I will add a block with library imports, then I will load the datasets and perform exploratory data analysis (EDA)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics

I will read the datasets and assign names, and while reading the **'orders'** and **'items'** datasets, I will change the date format to **'datetime'**.

In [2]:
url_customers = './olist_customers_dataset.csv'
url_items = './olist_order_items_dataset.csv'
url_orders = './olist_orders_dataset.csv'

customers = pd.read_csv(url_customers)
items = pd.read_csv(url_items, parse_dates=['shipping_limit_date'])
orders = pd.read_csv(url_orders, parse_dates=['order_purchase_timestamp', 
                                              'order_approved_at',
                                              'order_delivered_carrier_date',
                                              'order_delivered_customer_date',
                                              'order_estimated_delivery_date']
                    )

Let's delve into our datasets and check for missing values or interesting insights upon first glance

In [11]:
customers.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


In [4]:
customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [5]:
customers.isna().sum()   #using .isna to check number of missing values in columns

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [7]:
customers.nunique()   # this method return number of unique values

customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64

As we see: total value of customers = **96 096**

In [10]:
items.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


In [13]:
items.describe()   # The describe() function displays descriptive statistics for numerical columns"

,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653739,19.990320
std,0.705124,183.633928,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900000,13.080000
50%,1.000000,74.990000,16.260000
75%,1.000000,134.900000,21.150000
max,21.000000,6735.000000,409.680000


In [14]:
items.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [15]:
items.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [16]:
items.nunique()

order_id               98666
order_item_id             21
product_id             32951
seller_id               3095
shipping_limit_date    93318
price                   5968
freight_value           6999
dtype: int64

In [17]:
orders.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04


In [18]:
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [19]:
orders.nunique()

order_id                         99441
customer_id                      99441
order_status                         8
order_purchase_timestamp         98875
order_approved_at                90733
order_delivered_carrier_date     81018
order_delivered_customer_date    95664
order_estimated_delivery_date      459
dtype: int64

In [20]:
orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [21]:
pd.unique(orders['order_status']) # check unique status of orders

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

In [22]:
orders.order_status.unique()  # second way to check unique values in columns

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

## 1. How many users have made a single purchase?


Merge customers dataframe with orders, and assign variable:

In [23]:
customers_and_orders = customers.merge(orders, how = 'inner', on = 'customer_id') 
customers_and_orders.head(3)

# with method .merge, in parameters choose type of join and column or index level names to join on

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13


It's reasonable decision to consider only that 'purchase' will be the orders with status "delivered", and have a date in column order_delivered_customer_date. As a full cycle of deal.

In [24]:
customers_and_orders[['customer_unique_id','order_status', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']] \
                    .groupby('order_status',as_index = False) \
                    .count()

,order_status,customer_unique_id,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date
0,approved,2,2,0,0
1,canceled,625,484,75,6
2,created,5,0,0,0
3,delivered,96478,96464,96476,96470
4,invoiced,314,314,0,0
5,processing,301,301,0,0
6,shipped,1107,1107,1107,0
7,unavailable,609,609,0,0


In [26]:
buy = customers_and_orders \
                    .fillna(0) \
                    .query('order_status == "delivered"') \
                    .query('order_delivered_customer_date != 0') 
# filtering orders with status "delivered" and droping orders with empty date of delivery

Consider the number of buyers with orders equal to 1

In [27]:
buy.groupby('customer_unique_id', as_index = False) \
   .agg({'order_status': 'count'}) \
   .query('order_status == 1') \
   .nunique()

customer_unique_id    90549
order_status              1
dtype: int64

Answer - **90 549** customers, who made a single purchase.

## 2. On average, how many orders are not delivered due to various reasons (Make a detailed breakdown of reasons)?

Of all the order statuses available in the dataset, the **"canceled"** and **"unavailable"** statuses are the most suitable for the category of **"not delivered"**. All others statuses in one way or another precede the successful completion of the order delivery (or its cancellation) - accordingly, their value calculation is not taken.

First, let's look at the range of values in the order creation date column. To do this, we'll use the **max()** and **min()** functions.

In [30]:
print('Orders from {} to {}'.format(orders['order_purchase_timestamp'].min(),  # shows minimum value
                                    orders['order_purchase_timestamp'].max())) # shows maximum value

Orders from 2016-09-04 21:15:19 to 2018-10-17 17:30:18


As we can see, the first and last months in this dataset are incomplete. In order to perform correct monthly analysis, it is necessary to remove these incomplete data and only consider full months.

In [35]:
orders_month = orders[(orders['order_purchase_timestamp'] > '2016-10-01') & (orders['order_purchase_timestamp'] < '2018-09-30')]

In [36]:
orders_month['order_purchase_timestamp'] = orders_month.order_purchase_timestamp.dt.strftime('%Y-%m') 

# We use the dt.strftime method from the datetime library to transform the date format. 
# We need to remove the day data, leaving only the year and month


C:\Users\user\AppData\Local\Temp\ipykernel_6368\1174028165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_month['order_purchase_timestamp'] = orders_month.order_purchase_timestamp.dt.strftime('%Y-%m')


Let's check

In [37]:
orders_month['order_purchase_timestamp'].max()

'2018-09'

In [41]:
undelivered = (orders_month[orders_month["order_status"]
                .isin(["unavailable", "canceled"])]
                .groupby(['order_purchase_timestamp', 'order_status'], as_index = False) 
                .agg({'order_id': 'count'}) 
                .groupby('order_status') 
                .agg({'order_id': 'mean'}) 
                .rename(columns={'order_id':'avg'}) 
                .sort_values('avg', ascending=False)
              )
'''
1 We filter the necessary statuses using the isin() method.
2 We group by 2 columns, one of which must be order_status.
3 We count the number of values to determine the total number of orders by status.
4 Finding the mean and displaying the value after sorting in descending order
'''
undelivered

,avg
order_status,
unavailable,29.000000
canceled,28.136364


## 3. For each product, determine the day of the week on which the product is most frequently bought


To answer this question, I will use data from two datasets - one with information about products and one with information about orders (as it contains information about the order date). We will merge the two datasets into one and then convert the order creation date to the name of the day of the week.

In [44]:
orders_items = items.merge(orders, on = 'order_id') #объеденим таблицы с помощью .merge()


Next, we will take a closer look at the information stored in the data for one order with a large number of products, by filtering by the **order number** in the **"order_id"** column. As we can see, each individual product purchase (even if it is the same product in the order) is recorded in a separate row, so each row represents 1 item of the product. To make the calculations correct, I added a column with the number 1.

In [45]:
orders_items.query('order_id == "8272b63d03f5f79c56e9e4120aec44ef"')

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
57297,8272b63d03f5f79c56e9e4120aec44ef,1,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28
57298,8272b63d03f5f79c56e9e4120aec44ef,2,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28
57299,8272b63d03f5f79c56e9e4120aec44ef,3,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28
57300,8272b63d03f5f79c56e9e4120aec44ef,4,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28
57301,8272b63d03f5f79c56e9e4120aec44ef,5,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28
57302,8272b63d03f5f79c56e9e4120aec44ef,6,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28
57303,8272b63d03f5f79c56e9e4120aec44ef,7,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28
57304,8272b63d03f5f79c56e9e4120aec44ef,8,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28
57305,8272b63d03f5f79c56e9e4120aec44ef,9,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28
57306,8272b63d03f5f79c56e9e4120aec44ef,10,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,fc3d1daec319d62d49bfb5e1f83123e9,delivered,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28


In [46]:
orders_items['purchases'] = 1 

The purchase date is represented in the column "order_purchase_timestamp". We can convert it to the corresponding day names using the "dt.day_name" attribute of the datetime object, and store the data in a new column called "day".

In [48]:
orders_items['day'] = orders_items['order_purchase_timestamp'].dt.day_name()
orders_items.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purchases,day
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,1,Wednesday
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,1,Wednesday
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,1,Sunday


To find the maximum quantity of products purchased per day, you can use the **groupby()** function followed by the **nlargest()** method.





In [50]:
day_of_week = (orders_items
               .groupby(['product_id','day']) #группируем по 2 столбцам которые нас интересуют 
               .purchases.sum()  #суммируем по столбцу purchases для общей суммы по товару 
               .groupby(['product_id'])
               .nlargest(1) #параметр 1 указывает на количество строк, выводимой функцией, с наибольшим значением
              )

Output the obtained series with data on the quantity of purchased goods indicating the day of the week

In [51]:
day_of_week.sort_values(ascending = False).reset_index(level=[1,2]).head(10).reset_index(drop=True) 
#использовал функцию reset_index чтобы MultiIndex series конвертировать в датафрейм и убрать дублирующиеся столбцы

,product_id,day,purchases
0,422879e10f46682990de24d770e7f83d,Wednesday,93
1,99a4788cb24856965c36a24e339b6058,Monday,92
2,aca2eb7d00ea1a7b8ebd4e68314663af,Thursday,89
3,53b36df67ebb7c41585e8d54d6772e08,Tuesday,76
4,368c6c730842d78016ad823897a372db,Friday,67
5,389d119b48cf3043d311335e499d9c6b,Thursday,67
6,53759a2ecddad2bb87a079a1f1519f73,Wednesday,66
7,d1c427060a0f73f6b889a5c7c61f2ac4,Monday,64
8,3dd2a17168ec895c781a9191c1e95ad7,Wednesday,54
9,a62e25e09e05e6faf31d90c6ec1aa3d1,Thursday,50


## 4. What is the average number of purchases per customer per week (in month)? Please note that within a month, the number of weeks may not be a whole number. For example, November 2021 had 4.28 weeks
